# Periodic box on a turbulent flow in a 3D VEF plane channel with k-$\epsilon$ model.

## Introduction
 
 Validation made by: A.AUROUSSEAU (S.VANDROUX)
 Ported to jupyter notebook by Téo Boutin

### Description
 
This case test aims to validate the production of boundary conditions for an inlet/outlet turbulent flow in a plane channel, with the use of a periodic box.
k-$\epsilon$ modelling with wall function.
 Validation with comparison between profiles in the periodic box and in the main computation domain.
 


### TRUST parameters
 
* Version TRUST : 1.6.3 
* Type of problem : Hydraulique_Turbulent 
* Diffusion scheme : Default{} 
* Discretization : VEFPreP1B (ICEM mesh with prisms layer at walls) 
* Time scheme : Euler_implicite - Solveur Implicite 
* Solving of equations : "Navier Stokes turbulent" 
* Convection : EF_stab 
* Turbulence model : "k-$\epsilon$ model" 
* Wall law : "loi_standard_hydr" 
* Type of boundary conditions : Periodic box: periodicity in the x direction, walls in the y directions, symmetry in the z direction. Main computation domain: Imposed velocity field and K-Epsilon field (from periodic box) for the inlet, outlet pressure, walls and symmetry. 


In [ ]:
from trustutils import run
run.reset() 
run.initBuildDirectory()

In [ ]:
# prepare test case by running a periodic box

prepare=run.TRUSTCase("Calcul", "Prepare.data")
print("test")
if run.isExtractingNR():
    print("Prepare will only run on 3 time steps during non regression tests")
    prepare.substitute("# nb_pas_dt_max 3 #", "nb_pas_dt_max 3")
prepare.runCase(verbose=True)

# post pro for initialisation of main case
run.executeScript("postpro_prepare.sh")

In [ ]:
# run main case which starts from established velocity in Prepare.data

run.addCase("Calcul","mixedBC.data")

run.printCases()
run.runCases()

run.executeScript("postpro_mixedBC.sh")

### References: 
 
* Reichardt H. : Vollstaendige Darstellung der turbulenten Geschwindigkeitsverteliung in glatten Leitungen. ZAMM 31, 208-219 (1951) 
* http://www.sla.maschinenbau.tu-darmstadt.de/lehre/tms/Turbulence-TUDarmstadt-Pt2-6.pdf 


## Tests Description
 
Dimensions:
Lx=10 (main computational domain), Lx=0.4 (periodic box) , Ly=2.0 and Lz=0.8
 Initial conditions : Ux=10m/s
 Boundary conditions : the periodicity conditions need a source term to maintain the flow constant. For the main computation domain, a velocity field and a K-Epsilon field from the periodic box are applied on the inlet. The outlet is a fixed pressure
 Canal\_perio \{ bord PERIO \}
 The main computation mesh features prism layers at the walls and is generated with ICEM. The periodic box grid is generated from the main computation mesh using the Trio\_U keyword ExtrudeBord
 
 $\rho = 1 kg/m^3$
$\mu = 0.0001 Pa.s $ such as Reb = (Ub.h) / v = 100 000 (h=1 m : half-height)
 
![](src/./Geometry.png)

### Main Computation Mesh (13552 cells)
 
 

In [ ]:
from trustutils import visit
 
visu = visit.Show("Calcul/mixedBC.lata","Mesh","DOM_BODY")
visu.normal3D([0.2,0.3,0.9])
visu.up3D([-0.1,1,-0.3])
visu.zoom3D([0,0,0.8])
visu.visuOptions(["no_axes"])
visu.visuOptions(["no_bounding_box"])
visu.visuOptions(["no_databaseinfo"])
visu.visuOptions(["no_legend"])
visu.visuOptions(["no_triad"])
visu.plot()

### Periodic Box Mesh (2100 cells)
 
 

In [ ]:
from trustutils import visit
 
visu = visit.Show("Calcul/Prepare.lata","Mesh","dom_prepare")
visu.normal3D([0.2,0.3,0.9])
visu.up3D([-0.1,1,-0.3])
visu.zoom3D([0,0,0.8])
visu.visuOptions(["no_axes"])
visu.visuOptions(["no_bounding_box"])
visu.visuOptions(["no_databaseinfo"])
visu.visuOptions(["no_legend"])
visu.visuOptions(["no_triad"])
visu.plot()

## Results
 


In [ ]:
from trustutils import plot 
 
fig = plot.Graph(r"Computations convergence: friction velocity evolution") 
data = plot.loadText("./Calcul/u_tau.dat.perio")
fig.add((data[0]),(data[1]),label=r"Periodic Box",marker='+')
data = plot.loadText("./Calcul/u_tau.dat")
fig.add((data[0]),(data[1]),label=r"Main computation",marker='x')

fig.label(r"Time (s)",r"Friction velocity")

fig.visu(0,15.0,0.4,0.7)


In [ ]:
from trustutils import plot 
 
fig = plot.Graph(r"Velocity profiles") 
fig.addSegment("./Calcul/Prepare_SONDE_VIT.son",label=r"Periodic box established profile",marker='x') 
data = plot.loadText("./Calcul/mixedBC_SONDE_VIT.coupe.dat")
fig.add((data[0]),(data[1]),label=r"Main computation profile at x=1",marker='s')
data = plot.loadText("./Calcul/mixedBC_SONDE_VIT4.coupe.dat")
fig.add((data[0]),(data[1]),label=r"Main computation profile at x=9",marker='s')

fig.label(r"Y (m)",r"Vx Velocity (m/s)")

fig.visu(0.,2.,5.,12.)


In [ ]:
from trustutils import plot 
 
fig = plot.Graph(r"Dimensionless velocity profiles") 

import numpy as np 
x = np.linspace(10.0,10000.0,10000) 
y = 1/0.415*np.log(1+0.415*x)+7.44*(1-np.exp(-x/11)-(x)/11*np.exp(-x/3)) 
fig.add(x,y,label=r"Reichardt",marker='-') 
data = plot.loadText("./Calcul/Prepare_SONDE_VIT.coupe_dimensionless.dat")
fig.add((data[0]),(data[1]),label=r"Periodic box established profile",marker='x')
data = plot.loadText("./Calcul/mixedBC_SONDE_VIT4.coupe_dimensionless.dat")
fig.add((data[0]),(data[1]),label=r"Main computation profile at x=9",marker='s')

fig.label(r"Y+",r"U+")

fig.visu(10.,10000.,0.,30.)

fig.scale(xscale='log')


In [ ]:
from trustutils import plot 
 
fig = plot.Graph(r"Turbulent viscosity profiles") 
fig.addSegment("./Calcul/Prepare_SONDE_VISC_TURB.son",label=r"Periodic box  profile",marker='x') 
fig.addSegment("./Calcul/mixedBC_SONDE_VISC_TURB1.son",label=r"Main computation profile at x=1",marker='s') 
fig.addSegment("./Calcul/mixedBC_SONDE_VISC_TURB2.son",label=r"Main computation profile at x=0.9",marker='s') 

fig.label(r"y (m)",r"Nut")

fig.visu(0.,2.,0.,0.025)


In [ ]:
from trustutils import plot 
 
fig = plot.Graph(r"Dimensionless turbulent viscosity profiles") 
data = plot.loadText("./Calcul/mixedBC_SONDE_VISC_TURB1.coupe_Dimensionless.dat")
fig.add((data[0]),(data[1]),label=r"Profile at x=1",marker='x')
data = plot.loadText("./Calcul/mixedBC_SONDE_VISC_TURB2.coupe_Dimensionless.dat")
fig.add((data[0]),(data[1]),label=r"Profile at x=9",marker='s')
data = plot.loadText("nut_profile_LaunderSh.dat")
fig.add((data[0]),(data[1]),label=r"Launder-Sharma profile",marker='s')

fig.label(r"y (m)",r"Dimensionless Nut")

fig.visu(0.,1.,0.,0.15)


In [ ]:
from trustutils import plot 
 
fig = plot.Graph(r"Turbulence kinetic energy") 
fig.addSegment("./Calcul/Prepare_SONDE_K.son",label=r"Periodic box established profile",marker='x') 
fig.addSegment("./Calcul/mixedBC_SONDE_K1.son",label=r"Main computation profile at x=1",marker='s') 
fig.addSegment("./Calcul/mixedBC_SONDE_K4.son",label=r"Main computation profile at x=0.9",marker='s') 

fig.label(r"y (m)",r"K")

fig.visu(0.,2.,0.,0.7)


In [ ]:
from trustutils import plot 
 
fig = plot.Graph(r"Turbulence kinetic energy dissipation rate") 
fig.addSegment("./Calcul/Prepare_SONDE_EPS.son",label=r"Periodic box established profile",marker='x') 
fig.addSegment("./Calcul/mixedBC_SONDE_EPS1.son",label=r"Main computation profile at x=1",marker='s') 
fig.addSegment("./Calcul/mixedBC_SONDE_EPS4.son",label=r"Main computation profile at x=0.9",marker='s') 

fig.label(r"y (m)",r"Epsilon")

fig.visu(0.,2.,0.,7.0)


### $Re_{\tau}$ of the Periodic box computation
 
 

In [ ]:
from trustutils import plot 
 
columns=['y+ ', ' $Re_{\\tau}$ ', ' Theoretical $Re_{\\tau}$ ', ' Relative error (pourcent)'] 
tab = plot.Table(columns)
data = plot.loadText("./Calcul/ligneTableau.dat.perio",transpose=False, dtype="str")
tab.addLine([[(data[-1][0]),(data[-1][1]),(data[-1][3]),(data[-1][2]),]],r"Periodic Box")
tab.setTitle("$Re_{\tau}$ of the Periodic box computation")
display(tab)

### $Re_{\tau}$ of the main computation
 
 

In [ ]:
from trustutils import plot 
 
columns=['y+ ', ' $Re_{\\tau}$ ', ' Theoretical $Re_{\\tau}$ ', ' Relative error (pourcent)'] 
tab = plot.Table(columns)
data = plot.loadText("./Calcul/ligneTableau.dat",transpose=False, dtype="str")
tab.addLine([[(data[-1][0]),(data[-1][2]),(data[-1][3]),(data[-1][4]),]],r"Main computation")
tab.setTitle("$Re_{\tau}$ of the main computation")
display(tab)

## Conclusions
 


Use of inlet velocity and K-Eps fields extracted from a periodic box has proved to be usefull since we have good results on the velocity profiles and friction Reynolds, and the canal length needed to get an established flow is reduced
The velocity profiles of the main computation at two locations and the periodic box velocity profile are plotted on the same figure. The main computation profiles (one near the inlet: x=1.0 and one near the outlet: x=9.0) are relatively similar to each other. They're also close to the established profile of the periodic box.
 The dimensionless velocity profiles have similar slopes and slopes that are close to Reichardt curve.
 The dimensionless Nut profiles for the main computation are plotted, and they're quite far from the Launder-Sharma model profile.
 The friction Reynolds is also calculated in both computations. The error to the theory is much higher (6 pourcent) in the periodic box than in the main computation (0.08 pourcent). This difference on the friction Reynolds between the periodic box and the main computation domain can be explained by the difference in the meshing. Values of Y+ are indeed different.
 


## Performances
 


In [ ]:
from trustutils import plot 

# hack API because we cannot add cases from prepare step in perf table easily
columns = ["host", "system", "Total CPU Time", "CPU time/step", "number of cells"]
zeTable = plot.Table(columns)
prepare._addPerfToTable(zeTable)
for case in run.defaultSuite_.getCases():
    case._addPerfToTable(zeTable)
zeTable.sum("Total CPU Time")
zeTable.df


## Data Files

### mixedBC

In [ ]:
run.dumpDatasetMD("Calcul/mixedBC.data")